# Generate TMA run file

This automatically creates tiles which define FOVs on the MIBI slide based on user-set parameters for non-TMAs.

Notes about MIBI coordinate axes:

* The x-axis goes from left to right ascending
* The y-axis goes from bottom to top ascending

One limitation of the commercial instrument is that it's impossible to change FOV definitions once they're entered in. This presents a problem due in part to the nature of how a TMA input file is defined. The grid that the TMA defines leaves a lot of room for variation in between, and it is quite possible that a number of TMA-defined FOVs generated from `example_fov_grid_generate.ipynb` will not appear in the desired location on the TMA.

The goal is to avoid the "commercial instrument deadlock" with bad FOV definitions by being able to adjust these definitions beforehand. It should also be possible to verify the locations of these "redefined FOVs" compared to their TMA-defined FOV counterparts and adjust the names of the former so they match up as closely as possible to the desired scheme in the latter.

To accomplish this, after generating the tiles as defined by the TMA spec file, the script allows you to define a file listing the desired locations of each FOV on your TMA. We'll call these the manual FOVs and the FOVs generated by `example_fov_grid_generate.ipynb` the auto FOVs. Using this script, you can view how closely your manual FOV names match up with the auto FOV names and how close (or far) apart they are from each other. If needed, you can rematch as many manual FOV names as needed to different corresponding auto FOV names prior to remapping your manual FOVs.

In [ ]:
import json
import os
from skimage.io import imread

from ark.mibi import tiling_utils

# suppress mpl deprecation
import warnings
from matplotlib.cbook import mplDeprecation
warnings.filterwarnings("ignore", category=mplDeprecation)

# 1. Generate the FOVs from a TMA spec file

### Define paths to the JSON data

Define the following parameters to set your home directory:

* `base_dir`: the root directory of the tiling data
* `json_tiling_dir`: the directory in `base_dir` containing the information to read and write the FOV info

Define the prefix to use for your input and output file names:

* `tma_prefix`: defaults to `example_tma`

Define the following input and output paths (all prefixed by `fov_gen_prefix`):

* `fov_list_path`: the list of FOVs to use for tiling. Should contain 2 FOVs, one defining the upper-left corner and one defining the bottom-right corner FOV
* `manual_fov_path`: the path to your proposed set of FOVs. Make sure this file name is correct!
* `slide_path`: the path to the slide which to take the FOVs
* `mapping_path`: the path which to save the final mapping of your proposed FOVs to the FOVs generated by the script (the latter saved to `auto_fov_output_path.json`)
* `remapped_fov_path`: the path to write the tiles with remapped FOV names
* `moly_path`: the path to the Moly point, needed if you want to insert this between FOVs during remapping. Ignored if you choose not to insert Moly points.

In [ ]:
# define your home directory
base_dir = "../data/example_dataset"
json_tiling_dir = os.path.join(base_dir, "json_tiling_data")

# define the prefix of each file
tma_prefix = 'example_tma'

# define the input and output files
fov_list_path = os.path.join(json_tiling_dir, '%s_fov_list.json' % tma_prefix)
manual_fov_path = os.path.join(json_tiling_dir, '%s_proposed_fovs.json' % tma_prefix)
slide_path = os.path.join(json_tiling_dir, '%s_slide.png' % tma_prefix)
mapping_path = os.path.join(json_tiling_dir, '%s_manual_auto_map.json' % tma_prefix)
remapped_fov_path = os.path.join(json_tiling_dir, '%s_remapped_fovs.json' % tma_prefix)
moly_path = os.path.join(json_tiling_dir, '%s_moly_point.json' % tma_prefix)

### Define TMA grid dimensions

Define the following:

* `tma_num_x`: the number of FOVs along the x-axis to create. Note that you must define at least 3.
* `tma_num_y`: the number of FOVs along the y-axis to create. Note that you must define at least 3.

The FOVs created will be equally spaced along the coordinate grid defined by the top-left and bottom-right FOV in `fov_list_path`.

NOTE: randomization and Moly point insertion will be done after the names have been remapped in the section `Visualize and remap tiles`.

In [ ]:
tma_num_x = 4
tma_num_y = 7

### Create the FOVs on the TMA grid

Tiling will be done from top to bottom (descending y-axis order), then left to right (ascending x-axis order). FOV names correspond to their row and column position on the grid of FOVs and are 1-indexed.

For example, `R1C1` means the FOV is in row 1 and column 1 of the grid of FOVs, `R2C1` would be the next FOV down, and `R1C2` would be the next FOV to the right. In an unrandomized `auto_fov_regions`, `R2C1` would immediately follow `R1C1` and all of the `R{n}C1` FOVs would come before the `R{n}C2` FOVs for each region.

Note that due to the y-axis ascending from bottom to top, `R2C1's` y-axis coordinate is lower compared to `R1C1's`.

In [ ]:
auto_fov_regions = tiling_utils.tma_generate_fov_list(
    fov_list_path,
    tma_num_x,
    tma_num_y
)

# 2. Define the FOV locations and map to TMA-spec file FOV names

### Load the user-defined FOVs and slide image

In [ ]:
# load the user-defined set of FOVs in
with open(manual_fov_path, 'r') as mfop:
    manual_fov_regions = json.load(mfop)

In [ ]:
# load the slide image in
slide_data = imread(slide_path)

### Map proposed tiles to their closest automatically-generated tile (Euclidean)

In [ ]:
manual_to_auto_map, manual_fovs_info, auto_fovs_info = tiling_utils.assign_closest_fovs(
    manual_fov_regions,
    auto_fov_regions
)

### Visualize and remap tiles

Usage notes:

* Proposed FOVs are drawn in red. Automatically-generated FOVs are drawn in blue.
* The selected proposed FOV and its mapped automatically-generated FOV are colored a darker shade.
* The `Manually-defined FOV` tab can be used to visualize current mappings to automatically-generated FOVs.
* The `Automatically-generated FOV` tab is used for re-mapping a FOV created by this script to one of yours.
* When you're done generating your desired mapping, click the `Save mapping`. This will save your mapping to `mapping_path` specified earlier.

Note:

* The cells after this interactive widget (in the section `Use mapping to rename FOVs in tiled_regions_proposed`) are only to be run after you're satisfied with the mapping and clicked `Save mapping`.
* Prior to clicking `Save mapping` and re-running the cells in that section, ignore any error messages that may appear in that section.
* If you change your mind after saving the remapped FOVs, you can come back here to redo your mappings, save again, and re-run the code to remap the FOVs.

In [8]:
%matplotlib widget
tiling_utils.interactive_remap(
    manual_to_auto_map,
    manual_fovs_info,
    auto_fovs_info,
    slide_data,
    mapping_path,
    draw_radius=7,
    figsize=(7, 7)
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Output()

### Use mapping to rename FOVs in `manual_fov_regions`

In [ ]:
# load the mapping saved in the interactive visualization in
with open(mapping_path, 'r') as mp:
    mapping = json.load(mp)

In addition to renaming FOVs in `manual_fov_regions`, the following steps can be done if specified:

* The FOVs are randomized (set `randomize = True` in the following cell)
* Moly points are inserted at a specified interval (set `insert_moly = True` and `moly_interval` to the desired value in the following cell). Note that a Moly point will not be placed after the last FOV if the interval divides the number of FOVs equally.

In [ ]:
# whether to randomize the FOVs in remapped_fov_regions
randomize = False

# whether to insert Moly points between a specified interval of FOVs in remapped_fov_regions
moly_insert = True
moly_interval = 2

In [ ]:
# rename FOVs, randomize the order, and insert Moly points at a specified interval
remapped_fov_regions = tiling_utils.remap_and_reorder_fovs(
    manual_fov_regions,
    mapping,
    moly_path,
    randomize=randomize,
    moly_insert=moly_insert,
    moly_interval=moly_interval
)

In [ ]:
# save remapped_fov_regions with the new FOV names (and randomized and with Moly points inserted if specified)
with open(remapped_fov_path, 'w') as rtp:
    json.dump(remapped_fov_regions, rtp)